In [13]:
import pandas as pd
import folium

In [14]:
#Carrega o arquivo CSV com os dados e coordenadas
file_path_with_coords = '/home/gmelo/Área de trabalho/Gerador mapa/BD.csv'  # Substitua pelo caminho correto
df_coords = pd.read_csv(file_path_with_coords)

In [15]:
# Separar as coordenadas em colunas de latitude e longitude
df_coords[['Latitude', 'Longitude']] = df_coords['Latitude/Longitude'].str.split(',', expand=True)
df_coords['Latitude'] = df_coords['Latitude'].astype(float)
df_coords['Longitude'] = df_coords['Longitude'].astype(float)

In [16]:
# Remover linhas com valores NaN nas colunas de latitude ou longitude
df_coords = df_coords.dropna(subset=['Latitude', 'Longitude'])

In [17]:
# Função para definir o tamanho dos círculos conforme a quantidade de casos(índices), pode ser alterada de acordo com o proposito do projeto
def definir_tamanho(qnt):
    if qnt < 10:
        return 2
    elif qnt <= 25:
        return 4
    elif qnt <= 50:
        return 6
    else:
        return 9

In [18]:
#Gera o mapa
mapa = folium.Map(location=[-14.7900, -39.2800],
                  zoom_start=13,
                  tiles='cartodbpositron',
                  name='Stamen'
                  )

folium.TileLayer('cartodbpositron', name='Positon').add_to(mapa)
folium.TileLayer('cartodbdark_matter', name='Dark').add_to(mapa)
folium.TileLayer('https://{s}.tile-cyclosm.openstreetmap.fr/cyclosm/{z}/{x}/{y}.png',
                 name='Lagoon',
                 attr='<a href="https://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors').add_to(mapa)

folium.LayerControl().add_to(mapa)

In [19]:
# Adicionar os círculos e marcadores no mapa para SAC
for index, row in df_coords.iterrows():
    if row['qnt_sac'] > 0:
        folium.Circle(
            location=[row['Latitude'], row['Longitude']],
            radius=definir_tamanho(row['qnt_sac']) * 100,  # Multiplicador para melhor visualização
            color='red',
            fill=True,
            fill_color='red'
        ).add_to(mapa)

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            icon=folium.DivIcon(html=f"""<div style="display:none"></div>""")
        ).add_to(mapa).add_child(folium.Popup(f"Bairro: {row['Bairro']}<br>Quantidade SAC: {row['qnt_sac']}"))

In [20]:
# Adicionar os círculos e marcadores no mapa para SAI
for index, row in df_coords.iterrows():
    if row['qnt_sai'] > 0:
        folium.Circle(
            location=[row['Latitude'], row['Longitude']],
            radius=definir_tamanho(row['qnt_sai']) * 100,  # Multiplicador para melhor visualização
            color='yellow',
            fill=True,
            fill_color='yellow'
        ).add_to(mapa)

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            icon=folium.DivIcon(html=f"""<div style="display:none"></div>""")
        ).add_to(mapa).add_child(folium.Popup(f"Bairro: {row['Bairro']}<br>Quantidade SAI: {row['qnt_sai']}"))

In [21]:
# Adicionar uma legenda ao mapa
legend_html = '''
     <div style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 90px; 
     background-color: white;
     border:2px solid grey; z-index:9999; font-size:14px;
     ">
     &nbsp; <b>Legenda</b> <br>
     &nbsp; <i style="color:red;">●</i> SAC &nbsp; <br>
     &nbsp; <i style="color:yellow;">●</i> SAI &nbsp;
     </div>
     '''

mapa.get_root().html.add_child(folium.Element(legend_html))

In [22]:
# Salvar o mapa em um arquivo HTML
output_path = '/home/gmelo/Área de trabalho/Gerador mapa/mapa_insatisfatorios_com_coordenadas.html'# Altere o diretorio de output
mapa.save(output_path)

print(f"Mapa salvo em: {output_path}")

Mapa salvo em: /home/gmelo/Área de trabalho/Gerador mapa/mapa_insatisfatorios_com_coordenadas.html
